In [52]:
import cv2 
import numpy as np 
import pandas as pd 
import os
from sklearn import preprocessing 

In [109]:
raw =  {10:'left_ankel' , 13:'right_ankel' , 9:'left_knee',12:'right_knee',8:'left_hip',
        11:'right_hip', 4:'left_wrist',7:'right_wrist', 3:'left_elbow',6:'right_elbow',
        2:'left_shoulder',5:'right_shoulder',1:'nose',17:'right_ear',15:'right_eye',
        14:'left_eye' , 16:'left_ear', 0:'unknown'}
df_cols_to_keys = None 
sorted_keys = np.sort((np.array(list(raw.keys()))))
PART_IDS = {raw[i]:i for i in sorted_keys}

CONNECTED_PART_NAMES = [
    ("left_hip", "right_hip"), ("left_elbow", "left_shoulder"),
    ("left_elbow", "left_wrist"), ("left_hip", "left_knee"),
    ("left_knee", "left_ankel"), ("right_hip", "right_shoulder"),
    ("right_elbow", "right_shoulder"), ("right_elbow", "right_wrist"),
    ("right_hip", "right_knee"), ("right_knee", "right_ankel"),
    ("left_shoulder", "right_shoulder"), ("left_hip", "left_shoulder")
]
CONNECTED_PART_INDICES = [(PART_IDS[a], PART_IDS[b]) for a, b in CONNECTED_PART_NAMES];

UNCONNECTED_PART_NAMES = ["nose", "right_ear" , "left_ear", "right_eye" , "left_eye",  "unknown"]
UNCONNECTED_PART_INDICES = [PART_IDS[a] for a in UNCONNECTED_PART_NAMES];

ARGS = {'normDatasetPath':'/Users/sandeep/Desktop/dataandmodles/data/singlePlayersPosesL2Normalized.csv',
        'datasetPath':'/Users/sandeep/Desktop/dataandmodles/data/singlePlayersPoses.csv',
        'customDatasetPath' :  '/Users/sandeep/Desktop/dataandmodles/data',
        'datasetGen':False,
        'customDatasetGen': False,
        'customDatasetFrameSelectionList': [8]}

In [110]:
#get_custom_dataset(norm=False)

#Handling Main Dataset

In [111]:
def load_data():
    with open('/Users/sandeep/Downloads/dataset/annotation_dict.json') as f:
      data = json.load(f)
    labels = {0 : "block", 1 : "pass", 2 : "run", 
              3: "dribble",4: "shoot",5 : "ball in hand", 
              6 : "defense", 7: "pick" , 8 : "no_action" ,
              9: "walk" ,10: "discard"}
    return data , labels

In [112]:
if ARGS['datasetGen']:
    annotations , labels = load_data()
def get_label(fileName):
    label = annotations[fileName]
    return labels[label]

In [113]:
 def getScaleFactors(imgW, imgH, originalImgH , originalImgW):
    '''
       Helper function for get_data()
       Returns a tensor with given scalefactor (wdith,height). 
    '''
    return (imgW/originalImgW, imgH/originalImgH);

In [114]:
def mapToSquareImage(coord,scaleFactors):
    '''
      Helper function for get_data()
      Returns coord (width, height)
    '''
    coordInOriginalImage = [int(coord[0] * scaleFactors[0]) , int(coord[1] * scaleFactors[1])]
    return tuple(coordInOriginalImage)

In [115]:
def get_extended_pose(poses_dict, label=None):
    '''
    Helper function for get_data() and get_posenet_extended_pose()
    
    Parameters
    ----------
    pose : A dictionary containing 1 pose. The keys refer to bodyparts relative to the raw dict.
    Example {0:(x,y), ....} 
    
    label : get_data() will send label arugment but get_posenet_extended_pose() will not
    Return
    ---------
    extended_pose : A dictionary containing x and y coord has it own key
    Example {0_x: 1 ,0_y: 7 ....}  
    '''
    extended_pose = {}
    scale = getScaleFactors(244,244,176,128)
    for key,value in poses_dict.items():
        #TO DO: scale the value before setting to the value
        value = mapToSquareImage(value,scale)
        extended_pose[f'{key}_x'] = value[0]
        extended_pose[f'{key}_y'] = value[1]
    if label != None:
        extended_pose['label'] = label 
    return extended_pose
        

In [116]:
def filterPoseList(poseList , meidan=True):
    '''
    Helper function of get_data()
    
    '''
    if median:
        return np.array([poseList[7]])
    else:
        return poseList

In [117]:
def get_data():
    '''
    Helper function for generate_dataset()
    
    Returns
    -------
    data : A 1D list contining dict of 16 poses gotton from 16 frames for a single player
    Example [{0_x:1 , 0_y:2 , 1_x:67........,'label':'walk'] 
    The coordinates of the poses coords for players in sqaure image of size (244, 244)
    '''
    fileName = os.listdir('/Users/sandeep/Downloads/dataset/examples')
    fileName = [name for name in fileName if name[-1] != '4']
    data = []
    for name in fileName:
        posePath = f'/Users/sandeep/Downloads/dataset/examples/{name}'
        try:
            poses_list,label = np.load(posePath,allow_pickle=True) , get_label(name[0:-4])
            poses_list = filterPoseList(poses_list, meidan=True)
            for poses_dict in poses_list:
                extended_pose = get_extended_pose(pose_dict,label=label)
                data.append(extended_pose)
        except KeyError:
            continue
    return data   

In [118]:
def get_keys(df_keys):
    '''
    Helper function for get_proxy_coord
    Returns keys that the df columns name refers too. eg. 0_x -> 0 
    '''
    global df_cols_to_keys 
    return [df_cols_to_keys[key] for key in df_keys]
    

In [119]:
def get_connected_keys(key):
    key_pair_tuple = [t for t in CONNECTED_PART_INDICES if key in t]
    return [key1 if key1 != key else key2 for (key1,key2) in key_pair_tuple ]

In [120]:
def filter_keys(list1 , list2):
    '''
    Helper function for get_proxy_coord
    Returns keys that are not in the list2
    '''
    return list(set(list1)-set(list2))

In [121]:
def map_keys_to_df_key(key):
    return f'{key}_x' , f'{key}_y'

In [122]:
def get_unconneted_proxy_coord(keys):
    '''
    Helper function for get_proxy_coord
    '''
    usable_unconnected_keys = filter_keys(UNCONNECTED_PART_INDICES , keys)
    if len(usable_unconnected_keys) != 0:
        key_x , key_y  = map_keys_to_df_key(usable_unconnected_keys[0])
        return key_x , key_y
    else:
        return get_connected_proxy_coord(2,keys)#trace starts from right shoulder

In [123]:
def get_connected_proxy_coord(key,keys):
    '''
    Helper function for get_proxy_coord
    '''
    connected_keys_with_key = get_connected_keys(key)
    usable_connected_keys = filter_keys(connected_keys_with_key,keys)
    if  len(usable_connected_keys) != 0:#base-case, we know there is no pose with 1 points
        key_x , key_y = map_keys_to_df_key(usable_connected_keys[0])
        return key_x,key_y
    else:
       key_x, key_y = get_connected_proxy_coord(connected_keys_with_key[0],keys)
       return key_x,  key_y

In [124]:
def get_proxy_coord(row):
    '''
    Helper function to replace the NaN values/ the poses that are not visible using poxy poses
    PARAMS key: the key that gets passed is eg.1_x
    '''
    global df_cols_to_keys 
    #df keys with nan
    df_keys = [key for key in row[row.isnull()].index.tolist() if key[-1] != 'y'] 
    keys = get_keys(df_keys)
    for key in keys:
        if key in UNCONNECTED_PART_INDICES:#if the nan key is a  Unconnected part 
            key_x , key_y = get_unconneted_proxy_coord(keys)
        else:#if the nan key is a  Connected part 
            key_x, key_y = get_connected_proxy_coord(key,keys)
                
        current_key_x , current_key_y = map_keys_to_df_key(key)
        row[current_key_x] , row[current_key_y] = row[key_x] , row[key_y]
    return row

In [125]:
def get_df_cols():
    '''
    Helper function for generate_dataset()
    '''
    cols = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
    a = []
    for col in cols:
        xCol = f'{col}_x'
        yCol = f'{col}_y'   
        a.append(xCol)
        a.append(yCol)
    a.append('label')
    b = {}
    for col in cols:
        b[f'{col}_x'] = col
        b[f'{col}_y'] = col

    return a,b

In [126]:
def fillNaN(df):
    '''
    Helper function for generate_dataset()
    Updates the NaN cells with proxy coords. 
    '''
    return  df.apply(get_proxy_coord, axis=1)

In [127]:
def generate_dataset(data=None, norm=True):
    '''
    1.This function will fill the missing bodypoints with proxy coords and
      generate l2 normalized dataset from raw data
    
    2.Helper function for generate_data_posenet(). If norm = Flase is passed then l2 normalization
      is not applied.
    
    Parameters
    ----------
    data : A list with extended_poses.
    If data is not passed then you are aiming to generate the main pose dataset 
    Example : [{0_x:1, 0_y:1 ,17_y:16, 17_y:17} , {0_x:1, 0_y:1 ,16_y:10, 16_y:10} , {..}]
    
    Return
    -------
    normalized_data : A 2D array with normalized data 
    [[0_x, 0_y, 1_x, 1_y, .......17_y] , [......]]
    '''
    global df_cols_to_keys
    if ARGS['datasetGen']:
        data = get_data()
        df = pd.DataFrame(data)
    else:
      df = pd.DataFrame(data)
      df['0_x'] , df['0_y'] = np.nan , np.nan
        
    df_cols,df_cols_to_keysx = get_df_cols()
    df_cols_to_keys = df_cols_to_keysx
    df = df[df_cols[:-1]] #Generates nan values
    df = fillNaN(df)
    if norm:
        normalized_data = preprocessing.normalize(df, axis=1)
        return normalized_data
    else:
        unnormalized_data = df.to_numpy()
        return unnormalized_data
    

In [128]:
def read_dataset(name=None,norm=True):
    if name != None:
        return pd.read_csv(f"{ARGS['customDatasetPath']}/{name}.csv")
    elif norm:
        return pd.read_csv(ARGS['normDatasetPath'])
    elif norm == False:
        return pd.read_csv(ARGS['datasetPath'])



In [129]:
def get_custom_dataset(norm=True):
    '''
    Save custom dataset. You can select the frames you want in the ARGS. 
    The ARGS['customDatasetFrameSelectionList'] should refer to frame out of the 16 frames.
    Example name of the file:
        - custom_normalized_framelist_[7, 8, 9]_total_169530.csv
    '''
    df = read_dataset(norm=norm)
    condition = True
    for i,f in enumerate(ARGS['customDatasetFrameSelectionList']):
        if i == 0:
            if f == 8:
                condition =(((df.index+1)%f == 0) & ((df.index+1)%16 != 0))
            else:
                condition =  ((df.index+1)%f == 0)
        else:
             if f == 8:
                condition =condition | (((df.index+1)%f == 0) & ((df.index+1)%16 != 0))
             else:
                condition = condition | ((df.index+1)%f == 0)
    df = df[condition]
    fname = f'custom_normalized_{norm}_framelist_{ARGS["customDatasetFrameSelectionList"]}_total_{df.shape[0]}'
    df.to_csv(f'{ARGS["customDatasetPath"]}/{fname}.csv',index=False)

#Handeling Posenet Data

In [130]:
def posenet_part_to_part(part):
    '''
    Helper function of get_posenet_extended_pose
    '''
    if 'Ankle' in part:
        part = part.replace('Ankle' , 'ankel')
    if 'right' in part: 
        return f'right_{part[5:].lower()}'
    elif 'left' in part:
        return f'left_{part[4:].lower()}'
    return part
           

In [131]:
def map_part_to_key(posenetPart):
    '''
    Helper function of get_posenet_extended_pose()
    '''
    for key, part in raw.items():
        if part == posenetPart:
            return key
    raise Exception(f'{posenetPart} not in raw')

In [132]:
def get_posenet_extended_pose(pose):
    '''
    Helper function for generate generate_dataset()
    
    Parameters
    ----------
    pose : A list containing dictionary containing 1 pose. The keys refer to bodyparts relative to the posenet pred.
    Example [{'score': 0.8984865546226501, 'part': 'nose', 'position': {'x': 741.3767104497117, 'y': 201.7590852932459}}..].
    
    Return
    ---------
    extended_pose : A dictionary containing x and y coord has it own key
    Example {0_x: 1 ,0_y: 7 ....}  
    '''
    #Change keys
    pose_dict = {}
    for part_dict in pose:
        part = posenet_part_to_part(part_dict['part']) 
        key = map_part_to_key(part)
        pose_dict[key] = (part_dict['position']['x'] , part_dict['position']['y'])
    extended_pose = get_extended_pose(pose_dict)
    return extended_pose


In [133]:
def add_unkown_keypoint(pose):
    '''
    NOT USED ANYMORE! THIS FUNCTION DOES NOT RESPECT THE DATA PIPELINE
    Helper function of generate_data_posenet()
    This function appends a new  bodypart "unknown" object to the human keypoints list.
    This is important because all the poses in posenet pred will not contain thi body part, however,
    our original dataset contained this part.
    
    Parameter
    ---------
    pose : A 1D list with a human body parts
    Example [{score: , part:.. ,position:..}, secondBodyPart..a..]
    
    Return
    ----------
    pospose : A 1D list with a human body parts with 'unknown' body object 
    Example [{score: , part:'uknow' ,position:..} , {score: , part:.. ,position:..}, secondBodyPart..a..]
    '''
    unknow_obj = {'score': 0.1,
                 'part': 'unknown',
                 'position': {'x': np.nan, 'y': np.nan}}
    pose.append(unknow_obj)
    return pose

In [134]:
def generate_data_posenet(posenetPred,norm=True):
    '''
    This functions generates l2 normalized data form non-empty(Must have atleast 1 humans)
    Posenet results
    
    Parameters
    ---------
    posenetPred : The non-empty output of Posenet model
    Example {'detectionList': [{keypoints:[{score: , part:.. ,position:..}]} , human2 , ....]
    
    norm : True if you want a l2 normalized coordinates. False, if you want raw coordinate of poses 
            relative to image size (244,244)
    Return
    ------
    normalized_data : A 2D list with exte
    '''
    data = []
    for human in posenetPred['detectionList']:
        extended_pose = get_posenet_extended_pose(human['keypoints'])
        data.append(extended_pose)

    data = generate_dataset(data=data,norm=norm)
    return data
    

#Test 

In [137]:
generate_data_posenet(p,norm=False)

(10, 36)

In [13]:
#a = read_dataset(norm=True)

In [14]:
#a.columns.tolist()

['0_x',
 '0_y',
 '1_x',
 '1_y',
 '2_x',
 '2_y',
 '3_x',
 '3_y',
 '4_x',
 '4_y',
 '5_x',
 '5_y',
 '6_x',
 '6_y',
 '7_x',
 '7_y',
 '8_x',
 '8_y',
 '9_x',
 '9_y',
 '10_x',
 '10_y',
 '11_x',
 '11_y',
 '12_x',
 '12_y',
 '13_x',
 '13_y',
 '14_x',
 '14_y',
 '15_x',
 '15_y',
 '16_x',
 '16_y',
 '17_x',
 '17_y',
 'label']

In [34]:
#a.to_csv('/Users/sandeep/Desktop/dataandmodles/data/singlePlayersPoses.csv', index=False)
#df_normalized_df = pd.DataFrame(df_normalized, columns=df.columns)

In [22]:
#df_normalized_df

In [39]:
#df_normalized_df.to_csv('/Users/sandeep/Desktop/dataandmodles/data/singlePlayersPosesL2Normalized.csv', index=False)

In [1]:
x =np.load('/Users/sandeep/Downloads/dataset/examples/0000000_flipped.npy',allow_pickle=True) 

In [3]:
len(x)

16

#Test data

In [107]:
p = {'detectionList': [{'score': 0.9640490377650541,
   'keypoints': [{'score': 0.9522057771682739,
     'part': 'nose',
     'position': {'x': 750.0835971737026, 'y': 132.93605735025125}},
    {'score': 0.9521274566650391,
     'part': 'leftEye',
     'position': {'x': 752.3394106627999, 'y': 130.17028482658608}},
    {'score': 0.9105719327926636,
     'part': 'rightEye',
     'position': {'x': 748.8679435071575, 'y': 130.56975087673246}},
    {'score': 0.9261967539787292,
     'part': 'leftEar',
     'position': {'x': 757.6257090145878, 'y': 132.3716007973127}},
    {'score': 0.7733551263809204,
     'part': 'rightEar',
     'position': {'x': 747.0736848511622, 'y': 133.42985220467816}},
    {'score': 0.998728334903717,
     'part': 'leftShoulder',
     'position': {'x': 762.5365107246998, 'y': 160.18524975173392}},
    {'score': 0.9937459230422974,
     'part': 'rightShoulder',
     'position': {'x': 745.9086289045664, 'y': 150.48431374667214}},
    {'score': 0.9894365072250366,
     'part': 'leftElbow',
     'position': {'x': 772.3332924789257, 'y': 160.22679906412054}},
    {'score': 0.9791024923324585,
     'part': 'rightElbow',
     'position': {'x': 738.5586680246798, 'y': 152.54240566894953}},
    {'score': 0.9770278930664062,
     'part': 'leftWrist',
     'position': {'x': 773.2344003205889, 'y': 166.29253893543157}},
    {'score': 0.9824069738388062,
     'part': 'rightWrist',
     'position': {'x': 740.813940592324, 'y': 134.60338838798745}},
    {'score': 0.9998000860214233,
     'part': 'leftHip',
     'position': {'x': 755.5298068847846, 'y': 203.79839774021036}},
    {'score': 0.9995602369308472,
     'part': 'rightHip',
     'position': {'x': 742.8630143069149, 'y': 202.96940545688676}},
    {'score': 0.9976357221603394,
     'part': 'leftKnee',
     'position': {'x': 752.4640978647677, 'y': 235.63515374730736}},
    {'score': 0.9992009997367859,
     'part': 'rightKnee',
     'position': {'x': 737.9052723540498, 'y': 236.33655430911108}},
    {'score': 0.9804067611694336,
     'part': 'leftAnkle',
     'position': {'x': 751.9400137938214, 'y': 266.0537398055686}},
    {'score': 0.9773246645927429,
     'part': 'rightAnkle',
     'position': {'x': 739.1144989730416, 'y': 265.6878553023578}}]},
  {'score': 0.814132050556295,
   'keypoints': [{'score': 0.623327910900116,
     'part': 'nose',
     'position': {'x': 81.43360254618707, 'y': 277.2126254126352}},
    {'score': 0.05819198489189148,
     'part': 'leftEye',
     'position': {'x': 79.50436595673864, 'y': 274.18876915705266}},
    {'score': 0.6327947378158569,
     'part': 'rightEye',
     'position': {'x': 79.82234680994917, 'y': 273.95787459023074}},
    {'score': 0.26828086376190186,
     'part': 'leftEar',
     'position': {'x': 66.74411692720524, 'y': 274.23087607140764}},
    {'score': 0.8820042610168457,
     'part': 'rightEar',
     'position': {'x': 75.21089578835705, 'y': 275.103901082143}},
    {'score': 0.9929549694061279,
     'part': 'leftShoulder',
     'position': {'x': 127.94029447767468, 'y': 239.28392311099083}},
    {'score': 0.9995741844177246,
     'part': 'rightShoulder',
     'position': {'x': 74.06217343500639, 'y': 290.77925806690126}},
    {'score': 0.8475152254104614,
     'part': 'leftElbow',
     'position': {'x': 119.30687942457257, 'y': 257.4261059091657}},
    {'score': 0.9968534708023071,
     'part': 'rightElbow',
     'position': {'x': 74.90713828511899, 'y': 312.3998164238095}},
    {'score': 0.6821638941764832,
     'part': 'leftWrist',
     'position': {'x': 129.65270960970912, 'y': 275.8622102985349}},
    {'score': 0.9754737615585327,
     'part': 'rightWrist',
     'position': {'x': 83.11407396409395, 'y': 331.28246364229886}},
    {'score': 0.9980006217956543,
     'part': 'leftHip',
     'position': {'x': 128.28108106510766, 'y': 276.6535190569053}},
    {'score': 0.9981646537780762,
     'part': 'rightHip',
     'position': {'x': 65.81428357575567, 'y': 330.1056134572682}},
    {'score': 0.9977401494979858,
     'part': 'leftKnee',
     'position': {'x': 127.2138050879432, 'y': 315.4836628118875}},
    {'score': 0.9962053894996643,
     'part': 'rightKnee',
     'position': {'x': 63.30626659774303, 'y': 366.2762916935172}},
    {'score': 0.9460128545761108,
     'part': 'leftAnkle',
     'position': {'x': 116.29298754250362, 'y': 343.0726907273925}},
    {'score': 0.9449859261512756,
     'part': 'rightAnkle',
     'position': {'x': 51.5535710605045, 'y': 379.1055482173503}}]},
  {'score': 0.876895406666924,
   'keypoints': [{'score': 0.8928431272506714,
     'part': 'nose',
     'position': {'x': 573.9423286453466, 'y': 225.39207159005997}},
    {'score': 0.32192301750183105,
     'part': 'leftEye',
     'position': {'x': 572.6586261492097, 'y': 220.4433103723377}},
    {'score': 0.9254255294799805,
     'part': 'rightEye',
     'position': {'x': 571.6943271448847, 'y': 222.72911024837396}},
    {'score': 0.1245456337928772,
     'part': 'leftEar',
     'position': {'x': 563.0716834621929, 'y': 220.41000294809322}},
    {'score': 0.9872562289237976,
     'part': 'rightEar',
     'position': {'x': 563.2716462853249, 'y': 222.00218239028902}},
    {'score': 0.9332419633865356,
     'part': 'leftShoulder',
     'position': {'x': 560.8375331435757, 'y': 234.76994793460523}},
    {'score': 0.9992584586143494,
     'part': 'rightShoulder',
     'position': {'x': 557.3578911922992, 'y': 235.56939793459463}},
    {'score': 0.8153558969497681,
     'part': 'leftElbow',
     'position': {'x': 557.6467948608779, 'y': 261.33631133372074}},
    {'score': 0.9962073564529419,
     'part': 'rightElbow',
     'position': {'x': 549.4101474645283, 'y': 260.45745274429714}},
    {'score': 0.9543416500091553,
     'part': 'leftWrist',
     'position': {'x': 566.5737263421142, 'y': 277.23262170199916}},
    {'score': 0.9856947660446167,
     'part': 'rightWrist',
     'position': {'x': 551.6119831659076, 'y': 281.0885323734382}},
    {'score': 0.9982023239135742,
     'part': 'leftHip',
     'position': {'x': 540.2358658602473, 'y': 269.2878064398542}},
    {'score': 0.9991140365600586,
     'part': 'rightHip',
     'position': {'x': 538.0076421482881, 'y': 270.2474674971926}},
    {'score': 0.9966757297515869,
     'part': 'leftKnee',
     'position': {'x': 545.7901849728845, 'y': 300.9011559940913}},
    {'score': 0.9956839084625244,
     'part': 'rightKnee',
     'position': {'x': 550.833390238282, 'y': 302.10321788027545}},
    {'score': 0.9891136884689331,
     'part': 'leftAnkle',
     'position': {'x': 533.7035549416227, 'y': 327.93929263665314}},
    {'score': 0.9923385977745056,
     'part': 'rightAnkle',
     'position': {'x': 547.8488432185331, 'y': 336.78973011830027}}]},
  {'score': 0.7378928170484655,
   'keypoints': [{'score': 0.05663412809371948,
     'part': 'nose',
     'position': {'x': 316.3584126961811, 'y': 318.79641176220446}},
    {'score': 0.0076188743114471436,
     'part': 'leftEye',
     'position': {'x': 313.7953694840049, 'y': 315.753602350192}},
    {'score': 0.04753276705741882,
     'part': 'rightEye',
     'position': {'x': 319.1441268063663, 'y': 315.5706831107313}},
    {'score': 0.4059140682220459,
     'part': 'leftEar',
     'position': {'x': 307.2947993707121, 'y': 316.8341333068636}},
    {'score': 0.8818655014038086,
     'part': 'rightEar',
     'position': {'x': 320.74926896637, 'y': 318.0204503342018}},
    {'score': 0.9992328882217407,
     'part': 'leftShoulder',
     'position': {'x': 296.9083385967583, 'y': 334.2120851652254}},
    {'score': 0.9979104399681091,
     'part': 'rightShoulder',
     'position': {'x': 326.217355382874, 'y': 333.87021117978946}},
    {'score': 0.978861927986145,
     'part': 'leftElbow',
     'position': {'x': 292.33994674444494, 'y': 356.30654248083897}},
    {'score': 0.9961652755737305,
     'part': 'rightElbow',
     'position': {'x': 329.94422019644173, 'y': 357.77079564033386}},
    {'score': 0.9519994854927063,
     'part': 'leftWrist',
     'position': {'x': 292.70769338333946, 'y': 374.3261874891443}},
    {'score': 0.9950389862060547,
     'part': 'rightWrist',
     'position': {'x': 340.65342967429854, 'y': 377.613981159349}},
    {'score': 0.9991103410720825,
     'part': 'leftHip',
     'position': {'x': 303.701639815365, 'y': 374.9197370828334}},
    {'score': 0.9976000785827637,
     'part': 'rightHip',
     'position': {'x': 321.834617488542, 'y': 375.38668512523276}},
    {'score': 0.9744459390640259,
     'part': 'leftKnee',
     'position': {'x': 306.31318721134267, 'y': 407.8399305310571}},
    {'score': 0.9645605683326721,
     'part': 'rightKnee',
     'position': {'x': 328.21576616141977, 'y': 406.77084849570815}},
    {'score': 0.6179825663566589,
     'part': 'leftAnkle',
     'position': {'x': 302.84535558929156, 'y': 443.1752773335323}},
    {'score': 0.6717040538787842,
     'part': 'rightAnkle',
     'position': {'x': 326.17057605033807, 'y': 420.61816457817827}}]},
  {'score': 0.8599849676384645,
   'keypoints': [{'score': 0.8406587839126587,
     'part': 'nose',
     'position': {'x': 460.98564519417863, 'y': 147.40054474009176}},
    {'score': 0.49688032269477844,
     'part': 'leftEye',
     'position': {'x': 458.905171216948, 'y': 145.27062761515015}},
    {'score': 0.8656241297721863,
     'part': 'rightEye',
     'position': {'x': 459.0508587202627, 'y': 146.17679732139229}},
    {'score': 0.5785930752754211,
     'part': 'leftEar',
     'position': {'x': 446.77082346322084, 'y': 145.49839727180256}},
    {'score': 0.940955400466919,
     'part': 'rightEar',
     'position': {'x': 454.1038582833965, 'y': 147.4730739626562}},
    {'score': 0.9920663833618164,
     'part': 'leftShoulder',
     'position': {'x': 441.4927886164143, 'y': 152.52227939889997}},
    {'score': 0.9954123497009277,
     'part': 'rightShoulder',
     'position': {'x': 449.5946024538723, 'y': 156.24304277892747}},
    {'score': 0.7835468649864197,
     'part': 'leftElbow',
     'position': {'x': 442.000745454531, 'y': 165.9957985935839}},
    {'score': 0.9835329055786133,
     'part': 'rightElbow',
     'position': {'x': 450.5211255077119, 'y': 171.2341204904394}},
    {'score': 0.5842310786247253,
     'part': 'leftWrist',
     'position': {'x': 446.9847955953762, 'y': 180.78661994339066}},
    {'score': 0.9196459054946899,
     'part': 'rightWrist',
     'position': {'x': 454.732218753086, 'y': 186.54585676011425}},
    {'score': 0.9971888661384583,
     'part': 'leftHip',
     'position': {'x': 435.96177172571527, 'y': 178.37408384128085}},
    {'score': 0.9987317323684692,
     'part': 'rightHip',
     'position': {'x': 439.92045300730155, 'y': 181.05367714734794}},
    {'score': 0.994170069694519,
     'part': 'leftKnee',
     'position': {'x': 441.2504832563775, 'y': 202.1871326320919}},
    {'score': 0.9973212480545044,
     'part': 'rightKnee',
     'position': {'x': 446.1509831389238, 'y': 203.56296744173073}},
    {'score': 0.7612341046333313,
     'part': 'leftAnkle',
     'position': {'x': 432.8612382938799, 'y': 224.54999905194524}},
    {'score': 0.889951229095459,
     'part': 'rightAnkle',
     'position': {'x': 441.6587074169803, 'y': 229.49203060817882}}]},
  {'score': 0.5040977719952079,
   'keypoints': [{'score': 0.4599902033805847,
     'part': 'nose',
     'position': {'x': 136.8994536694516, 'y': 225.15386402916863}},
    {'score': 0.1653466522693634,
     'part': 'leftEye',
     'position': {'x': 134.92039266076128, 'y': 222.72322419693813}},
    {'score': 0.5148405432701111,
     'part': 'rightEye',
     'position': {'x': 136.43569359916276, 'y': 222.88832293267473}},
    {'score': 0.6765022873878479,
     'part': 'leftEar',
     'position': {'x': 128.7941680865341, 'y': 224.07977849606624}},
    {'score': 0.860223114490509,
     'part': 'rightEar',
     'position': {'x': 136.18710938836452, 'y': 224.309426550642}},
    {'score': 0.9929549694061279,
     'part': 'leftShoulder',
     'position': {'x': 122.70406450373997, 'y': 239.85546343743903}},
    {'score': 0.9964224100112915,
     'part': 'rightShoulder',
     'position': {'x': 140.80864582466572, 'y': 240.09921547114538}},
    {'score': 0.8475152254104614,
     'part': 'leftElbow',
     'position': {'x': 119.30687942457257, 'y': 257.4261059091657}},
    {'score': 0.9881073236465454,
     'part': 'rightElbow',
     'position': {'x': 142.69865219959158, 'y': 262.31026015124655}},
    {'score': 0.6821638941764832,
     'part': 'leftWrist',
     'position': {'x': 129.65270960970912, 'y': 275.8622102985349}},
    {'score': 0.9227277040481567,
     'part': 'rightWrist',
     'position': {'x': 150.09683187534745, 'y': 283.6282689906075}},
    {'score': 0.9980006217956543,
     'part': 'leftHip',
     'position': {'x': 128.28108106510766, 'y': 276.6535190569053}},
    {'score': 0.9985337257385254,
     'part': 'rightHip',
     'position': {'x': 138.23992268423015, 'y': 275.4075156068058}},
    {'score': 0.9977401494979858,
     'part': 'leftKnee',
     'position': {'x': 127.2138050879432, 'y': 315.4836628118875}},
    {'score': 0.9974015951156616,
     'part': 'rightKnee',
     'position': {'x': 148.92833325151497, 'y': 312.57790108983187}},
    {'score': 0.9460128545761108,
     'part': 'leftAnkle',
     'position': {'x': 116.29298754250362, 'y': 343.0726907273925}},
    {'score': 0.9895665645599365,
     'part': 'rightAnkle',
     'position': {'x': 151.31664061814212, 'y': 344.80584357970713}}]},
  {'score': 0.8883470963029301,
   'keypoints': [{'score': 0.9016766548156738,
     'part': 'nose',
     'position': {'x': 191.87520541501848, 'y': 224.33076664810574}},
    {'score': 0.9343790411949158,
     'part': 'leftEye',
     'position': {'x': 215.40888025519553, 'y': 202.1297436305927}},
    {'score': 0.9898753762245178,
     'part': 'rightEye',
     'position': {'x': 209.024632289615, 'y': 203.6690314282991}},
    {'score': 0.8741616010665894,
     'part': 'leftEar',
     'position': {'x': 217.49377976940215, 'y': 200.95465830004605}},
    {'score': 0.9843364357948303,
     'part': 'rightEar',
     'position': {'x': 207.47539403286765, 'y': 200.73050281451935}},
    {'score': 0.9610521793365479,
     'part': 'leftShoulder',
     'position': {'x': 186.44536705350458, 'y': 237.59781582268727}},
    {'score': 0.9832644462585449,
     'part': 'rightShoulder',
     'position': {'x': 179.70832343702756, 'y': 231.6079320725783}},
    {'score': 0.8181886672973633,
     'part': 'leftElbow',
     'position': {'x': 198.75025332494917, 'y': 241.4248808971517}},
    {'score': 0.9742301106452942,
     'part': 'rightElbow',
     'position': {'x': 176.64679099855647, 'y': 248.60958883460717}},
    {'score': 0.7595099806785583,
     'part': 'leftWrist',
     'position': {'x': 212.3417351873924, 'y': 239.54550349562584}},
    {'score': 0.9742575883865356,
     'part': 'rightWrist',
     'position': {'x': 165.79313145743475, 'y': 276.2488039618348}},
    {'score': 0.9755503535270691,
     'part': 'leftHip',
     'position': {'x': 187.3309318790126, 'y': 266.3152493124818}},
    {'score': 0.9934101104736328,
     'part': 'rightHip',
     'position': {'x': 177.1336840630768, 'y': 263.0113274162729}},
    {'score': 0.9868189096450806,
     'part': 'leftKnee',
     'position': {'x': 197.71123682515002, 'y': 276.13066567994485}},
    {'score': 0.9921814203262329,
     'part': 'rightKnee',
     'position': {'x': 184.4295162386662, 'y': 283.96034890609553}},
    {'score': 0.997875988483429,
     'part': 'leftAnkle',
     'position': {'x': 184.06187586123576, 'y': 308.53284727886603}},
    {'score': 0.9753893613815308,
     'part': 'rightAnkle',
     'position': {'x': 184.4661551914858, 'y': 313.93272669641766}}]},
  {'score': 0.7813626632970923,
   'keypoints': [{'score': 0.12348228693008423,
     'part': 'nose',
     'position': {'x': 688.8696678866459, 'y': 265.29690829761313}},
    {'score': 0.04080009460449219,
     'part': 'leftEye',
     'position': {'x': 687.3506835337434, 'y': 262.60347837054746}},
    {'score': 0.0938519835472107,
     'part': 'rightEye',
     'position': {'x': 689.4395390104562, 'y': 262.6860671878067}},
    {'score': 0.5556731224060059,
     'part': 'leftEar',
     'position': {'x': 682.720598805412, 'y': 263.7090091143365}},
    {'score': 0.7531518340110779,
     'part': 'rightEar',
     'position': {'x': 694.4195149096656, 'y': 263.30416833755254}},
    {'score': 0.9926143884658813,
     'part': 'leftShoulder',
     'position': {'x': 676.0017759939852, 'y': 281.58802488317104}},
    {'score': 0.9975892305374146,
     'part': 'rightShoulder',
     'position': {'x': 701.5218682354606, 'y': 281.84680226687954}},
    {'score': 0.9812663793563843,
     'part': 'leftElbow',
     'position': {'x': 668.1287872062045, 'y': 309.01805582162206}},
    {'score': 0.9886804819107056,
     'part': 'rightElbow',
     'position': {'x': 705.5622934253325, 'y': 310.9837678912607}},
    {'score': 0.8687466979026794,
     'part': 'leftWrist',
     'position': {'x': 663.4013290262401, 'y': 328.3890470508066}},
    {'score': 0.9612067937850952,
     'part': 'rightWrist',
     'position': {'x': 705.2921039186018, 'y': 335.315051698602}},
    {'score': 0.9938793182373047,
     'part': 'leftHip',
     'position': {'x': 680.020238393999, 'y': 332.3338394760052}},
    {'score': 0.9815400838851929,
     'part': 'rightHip',
     'position': {'x': 697.0172960064682, 'y': 332.548811345621}},
    {'score': 0.9961481094360352,
     'part': 'leftKnee',
     'position': {'x': 679.985505316439, 'y': 364.2498189638557}},
    {'score': 0.9929929971694946,
     'part': 'rightKnee',
     'position': {'x': 701.6961643609512, 'y': 368.4696156222444}},
    {'score': 0.9779824614524841,
     'part': 'leftAnkle',
     'position': {'x': 679.9862640106425, 'y': 401.1853981315573}},
    {'score': 0.9835590124130249,
     'part': 'rightAnkle',
     'position': {'x': 705.2554826760262, 'y': 408.61782801378536}}]},
  {'score': 0.40536051988601685,
   'keypoints': [{'score': 0.9832150936126709,
     'part': 'nose',
     'position': {'x': 213.18634583262468, 'y': 204.86075359118047}},
    {'score': 0.9343790411949158,
     'part': 'leftEye',
     'position': {'x': 215.40888025519553, 'y': 202.1297436305927}},
    {'score': 0.9898753762245178,
     'part': 'rightEye',
     'position': {'x': 209.024632289615, 'y': 203.6690314282991}},
    {'score': 0.8741616010665894,
     'part': 'leftEar',
     'position': {'x': 217.49377976940215, 'y': 200.95465830004605}},
    {'score': 0.9843364357948303,
     'part': 'rightEar',
     'position': {'x': 207.47539403286765, 'y': 200.73050281451935}},
    {'score': 0.959257960319519,
     'part': 'leftShoulder',
     'position': {'x': 226.494074165449, 'y': 203.7482727371427}},
    {'score': 0.9969682097434998,
     'part': 'rightShoulder',
     'position': {'x': 205.9036569351263, 'y': 203.26983330526633}},
    {'score': 0.9914251565933228,
     'part': 'leftElbow',
     'position': {'x': 235.3801564628563, 'y': 220.4784648372974}},
    {'score': 0.9885672330856323,
     'part': 'rightElbow',
     'position': {'x': 201.13948238327558, 'y': 219.62597765996924}},
    {'score': 0.996604323387146,
     'part': 'leftWrist',
     'position': {'x': 238.71615296520395, 'y': 230.80469088496534}},
    {'score': 0.9648628234863281,
     'part': 'rightWrist',
     'position': {'x': 198.00437425406236, 'y': 240.6558707805812}},
    {'score': 0.9936432838439941,
     'part': 'leftHip',
     'position': {'x': 213.38116357686664, 'y': 243.2344267364184}},
    {'score': 0.9964041709899902,
     'part': 'rightHip',
     'position': {'x': 208.19243348939588, 'y': 237.2323637454927}},
    {'score': 0.9868189096450806,
     'part': 'leftKnee',
     'position': {'x': 197.71123682515002, 'y': 276.13066567994485}},
    {'score': 0.9937173128128052,
     'part': 'rightKnee',
     'position': {'x': 195.73707744274543, 'y': 274.7723536069926}},
    {'score': 0.9764502048492432,
     'part': 'leftAnkle',
     'position': {'x': 191.2328427204032, 'y': 304.7070607692778}},
    {'score': 0.9718047380447388,
     'part': 'rightAnkle',
     'position': {'x': 190.33239724186623, 'y': 309.3275158103665}}]},
  {'score': 0.9040610229267794,
   'keypoints': [{'score': 0.8910136818885803,
     'part': 'nose',
     'position': {'x': 670.4615441154451, 'y': 160.74282339747276}},
    {'score': 0.9184383153915405,
     'part': 'leftEye',
     'position': {'x': 670.8869036961435, 'y': 158.85383482987586}},
    {'score': 0.8340492248535156,
     'part': 'rightEye',
     'position': {'x': 668.9841424034776, 'y': 159.6214339985178}},
    {'score': 0.8678550720214844,
     'part': 'leftEar',
     'position': {'x': 674.6975455540098, 'y': 161.00397027344596}},
    {'score': 0.774032711982727,
     'part': 'rightEar',
     'position': {'x': 667.4519846650693, 'y': 162.06414939047352}},
    {'score': 0.9905576705932617,
     'part': 'leftShoulder',
     'position': {'x': 675.923336311226, 'y': 170.3972505936383}},
    {'score': 0.9952820539474487,
     'part': 'rightShoulder',
     'position': {'x': 664.9722843432098, 'y': 170.81160626006496}},
    {'score': 0.957453191280365,
     'part': 'leftElbow',
     'position': {'x': 679.2347778369685, 'y': 174.48536467924066}},
    {'score': 0.9744076728820801,
     'part': 'rightElbow',
     'position': {'x': 650.2636407942658, 'y': 178.9483243489389}},
    {'score': 0.9598407745361328,
     'part': 'leftWrist',
     'position': {'x': 676.4268711860409, 'y': 179.63972803046425}},
    {'score': 0.9672783613204956,
     'part': 'rightWrist',
     'position': {'x': 647.7886223019136, 'y': 182.38415135296006}},
    {'score': 0.9503251314163208,
     'part': 'leftHip',
     'position': {'x': 668.8788241363792, 'y': 200.6021992337972}},
    {'score': 0.9670970439910889,
     'part': 'rightHip',
     'position': {'x': 659.9120041255499, 'y': 200.4408972490598}},
    {'score': 0.8634660243988037,
     'part': 'leftKnee',
     'position': {'x': 665.1070198316252, 'y': 219.68501822539383}},
    {'score': 0.9344547986984253,
     'part': 'rightKnee',
     'position': {'x': 652.3500117767467, 'y': 221.53708620914347}},
    {'score': 0.6912509202957153,
     'part': 'leftAnkle',
     'position': {'x': 648.2078462950745, 'y': 239.74370856012376}},
    {'score': 0.8322347402572632,
     'part': 'rightAnkle',
     'position': {'x': 645.144524133663, 'y': 238.4149075397379}}]}]}